# Getting Started with AutoGen

## Welcome to AutoGen!

AutoGen is a programming framework for building multi-agent AI applications. In this quickstart notebook, we'll:

1. Install AutoGen
2. Create your first agent
3. Build a simple two-agent conversation
4. Add tools and function calling
5. Stream agent responses

Let's get started!

## 1. Installation

First, let's install the required packages:

In [ ]:
# Install AutoGen AgentChat and Extensions
!pip install -U "autogen-agentchat" "autogen-ext[openai]"

## 2. Setting up API Keys

AutoGen works with various LLM providers. For this tutorial, we'll use OpenAI:

In [1]:
import os
from getpass import getpass

# Set your OpenAI API key
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

## 3. Creating Your First Agent

Let's create a simple assistant agent:

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Initialize the model client
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    # api_key=os.environ["OPENAI_API_KEY"],  # Optional, will use env var
)

# Create an assistant agent
assistant = AssistantAgent(
    name="helpful_assistant",
    model_client=model_client,
    system_message="You are a helpful AI assistant. Be concise and informative.",
)

print("Agent created successfully!")

Agent created successfully!


## 4. Running Your First Agent Task

Now let's run a simple task with our agent:

In [3]:
# Run a simple task
result = await assistant.run(task="What is a large language model?")

# Print the result
print("Agent Response:")
print("-" * 50)
for message in result.messages:
    print(f"{message.source}: {message.content}")

Agent Response:
--------------------------------------------------
user: What is a large language model?
helpful_assistant: A large language model (LLM) is a type of artificial intelligence model designed to understand, generate, and manipulate human language. LLMs are typically based on deep learning architectures, particularly transformer networks. They are trained on vast datasets containing text from books, websites, and other written sources, allowing them to learn patterns, grammar, and contextual relationships in language.

Key characteristics of LLMs include:

1. **Size**: They have millions to billions of parameters, which are the variables that the model learns from the training data.
2. **Contextual Understanding**: LLMs can understand context, allowing them to produce coherent and relevant responses.
3. **Versatility**: They can perform various tasks such as translation, summarization, question answering, and creative writing.

Examples include OpenAI's GPT-3 and GPT-4, Goo

## 5. Adding Tools to Your Agent

Agents become more powerful when they can use tools. Let's add a simple tool:

In [4]:
# Define a simple tool
async def get_weather(city: str) -> str:
    """Get the weather for a given city.
    
    Args:
        city: The name of the city
    
    Returns:
        A string describing the weather
    """
    # This is a mock function for demonstration
    weather_data = {
        "New York": "Sunny, 72°F",
        "London": "Cloudy, 59°F",
        "Tokyo": "Rainy, 65°F",
        "Paris": "Partly cloudy, 68°F",
    }
    return weather_data.get(city, f"Weather data not available for {city}")

# Create an agent with tools
weather_agent = AssistantAgent(
    name="weather_assistant",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a weather assistant. Use the get_weather tool to provide weather information.",
)

In [5]:
# Run a task that uses the tool
result = await weather_agent.run(task="What's the weather like in New York and Paris?")

print("Weather Agent Response:")
print("-" * 50)
for message in result.messages:
    if message.content:  # Only print messages with content
        print(f"{message.source}: {message.content}")

Weather Agent Response:
--------------------------------------------------
user: What's the weather like in New York and Paris?
weather_assistant: [FunctionCall(id='call_EdgfQ0QZMWezlSeeV2O1on4E', arguments='{"city": "New York"}', name='get_weather'), FunctionCall(id='call_oI3b3vzE9tpxVyPb4UzSULu5', arguments='{"city": "Paris"}', name='get_weather')]
weather_assistant: [FunctionExecutionResult(content='Sunny, 72°F', name='get_weather', call_id='call_EdgfQ0QZMWezlSeeV2O1on4E', is_error=False), FunctionExecutionResult(content='Partly cloudy, 68°F', name='get_weather', call_id='call_oI3b3vzE9tpxVyPb4UzSULu5', is_error=False)]
weather_assistant: Sunny, 72°F
Partly cloudy, 68°F


## 6. Creating a Two-Agent Conversation

Let's create a conversation between two agents:

In [20]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Initialize the model client properly
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
)

# Create a writer agent
writer = AssistantAgent(
    name="writer",
    model_client=model_client,
    system_message="You are a creative writer. Write short, engaging content.",
)

# Create a critic agent
critic = AssistantAgent(
    name="critic",
    model_client=model_client,
    system_message="You are a constructive critic. Provide brief, helpful feedback to improve the writing. After 2 rounds of feedback, say 'APPROVED' if satisfied.",
)

# Create a team with both agents
team = RoundRobinGroupChat(
    participants=[writer, critic],
    termination_condition=MaxMessageTermination(max_messages=6),
)

output = await team.run(task="Write a haiku about artificial intelligence")

for message in output.messages:
    print(f"\n{message.source}:")
    print(message.content)


user:
Write a haiku about artificial intelligence

writer:
Silicon dreams weave light,  
Thoughts dance in electric winds,  
Mind of circuits blooms.

critic:
This haiku captures the essence of artificial intelligence beautifully. However, to enhance the imagery and emotional depth, consider using a more vivid metaphor or specific imagery related to AI. 

For example, instead of "thoughts dance," you could use a more dynamic verb that evokes action, like "ideas spark" or "knowledge flows." 

Overall, great start! Make these tweaks to elevate your poem.

writer:
Thank you for the thoughtful critique! Here’s a revised version of the haiku with your suggestions in mind:

Silicon dreams ignite,  
Ideas spark in circuits' flow,  
Mind of stars unfolds.

critic:
This revision is fantastic! The phrase "Ideas spark in circuits' flow" delivers a more dynamic image, and "Mind of stars unfolds" adds a touch of wonder. To enhance clarity and rhythm, consider focusing on a stronger first line that

## 7. Streaming Agent Responses

For better user experience, we can stream agent responses:

In [21]:
from autogen_agentchat.ui import Console

# Create an agent with streaming enabled
streaming_agent = AssistantAgent(
    name="streaming_assistant",
    model_client=model_client,
    system_message="You are a helpful assistant. Provide detailed explanations.",
    model_client_stream=True,  # Enable streaming
)

# Stream the response to console
await Console(
    streaming_agent.run_stream(
        task="Explain the concept of an llm based agent in 3 key points"
    )
)

---------- TextMessage (user) ----------
Explain the concept of an llm based agent in 3 key points
---------- ModelClientStreamingChunkEvent (streaming_assistant) ----------
Certainly! Here are three key points that explain the concept of an LLM-based agent:

1. **Natural Language Processing Capabilities**:
   LLM-based agents utilize large language models (LLMs) to understand and generate human-like text. This allows them to engage in natural language processing tasks, such as answering questions, providing explanations, and even carrying on conversations. The underlying LLM is trained on vast amounts of text data, enabling it to grasp context, nuance, and various aspects of human language, making it effective for a wide range of applications.

2. **Task Automation and Assistance**:
   These agents can automate various tasks by interacting with users in a conversational manner. They can assist in information retrieval, scheduling, writing, and many other functions across different dom

TaskResult(messages=[TextMessage(id='0b9d538e-d475-49ed-a068-9ca24a744678', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 20, 13, 45, 19, 388286, tzinfo=datetime.timezone.utc), content='Explain the concept of an llm based agent in 3 key points', type='TextMessage'), TextMessage(id='8826ec1b-241c-4a41-95cb-fb92a7d64b8b', source='streaming_assistant', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, created_at=datetime.datetime(2025, 8, 20, 13, 45, 31, 752821, tzinfo=datetime.timezone.utc), content='Certainly! Here are three key points that explain the concept of an LLM-based agent:\n\n1. **Natural Language Processing Capabilities**:\n   LLM-based agents utilize large language models (LLMs) to understand and generate human-like text. This allows them to engage in natural language processing tasks, such as answering questions, providing explanations, and even carrying on conversations. The underlying LLM is trained on vas

## 8. Code Execution with Agents

AutoGen agents can also execute code:

In [22]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.agents import CodeExecutorAgent, ApprovalRequest, ApprovalResponse
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from pathlib import Path
import venv
import tempfile

def simple_approval_func(request: ApprovalRequest) -> ApprovalResponse:
    """Simple approval function that requests user input for code execution approval."""
    print("Code execution approval requested:")
    print("=" * 50)
    print(request.code)
    print("=" * 50)

    while True:
        user_input = input("Do you want to execute this code? (y/n): ").strip().lower()
        if user_input in ['y', 'yes']:
            return ApprovalResponse(approved=True, reason='Approved by user')
        elif user_input in ['n', 'no']:
            return ApprovalResponse(approved=False, reason='Denied by user')
        else:
            print("Please enter 'y' for yes or 'n' for no.")

# Create a temporary directory for code execution
temp_dir = tempfile.mkdtemp()

# Create a code-writing assistant
coder = AssistantAgent(
    name="coder",
    model_client=model_client,
    system_message="You are a Python programmer. Write clean, simple code. Always include print statements to show results.",
)

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)
venv_dir = work_dir / ".venv"
venv_builder = venv.EnvBuilder(with_pip=True)
venv_builder.create(venv_dir)
venv_context = venv_builder.ensure_directories(venv_dir)

local_code_executor = LocalCommandLineCodeExecutor(work_dir=work_dir, virtual_env_context=venv_context)

code_executor_agent = CodeExecutorAgent(
    "code_executor",
    code_executor=local_code_executor,
    approval_func=simple_approval_func
)

# Create a team
coding_team = RoundRobinGroupChat(
    participants=[coder, code_executor_agent],
    termination_condition=MaxMessageTermination(max_messages=4),
)

# Run a coding task
result = await coding_team.run(
    task="Write a Python function to calculate the Fibonacci sequence up to n=10 and print the results"
)

print("\nCoding Session:")
print("=" * 50)
for message in result.messages:
    print(f"\n[{message.source}]:")
    print(message.content[:500] if len(message.content) > 500 else message.content)

Code execution approval requested:
```python
def fibonacci_sequence(n):
    fib_sequence = []
    a, b = 0, 1
    for _ in range(n):
        fib_sequence.append(a)
        a, b = b, a + b
    return fib_sequence

n = 10
result = fibonacci_sequence(n)
print("Fibonacci sequence up to n =", n, ":", result)

```

Coding Session:

[user]:
Write a Python function to calculate the Fibonacci sequence up to n=10 and print the results

[coder]:
Here is a simple Python function that calculates the Fibonacci sequence up to \( n = 10 \) and prints the results:

```python
def fibonacci_sequence(n):
    fib_sequence = []
    a, b = 0, 1
    for _ in range(n):
        fib_sequence.append(a)
        a, b = b, a + b
    return fib_sequence

n = 10
result = fibonacci_sequence(n)
print("Fibonacci sequence up to n =", n, ":", result)
```

When you run this code, it will output the Fibonacci sequence up to the 10th number.

[code_executor]:
Fibonacci sequence up to n = 10 : [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

/var/folders/5l/y8s3fc655417629rqwgxkhx80000gn/T/ipykernel_29400/3161963065.py:56: RuntimeWarning: coroutine 'BaseGroupChat.run' was never awaited
  result = await coding_team.run(


In [25]:
result = await coding_team.run(
    task="Write a Python function to display a triangle with string characters."
)

print("\nCoding Session:")
print("=" * 50)
for message in result.messages:
    print(f"\n[{message.source}]:")
    print(message.content[:500] if len(message.content) > 500 else message.content)

Code execution approval requested:
```python
def print_pattern(rows):
    for i in range(rows):
        print(' ' * (rows - i - 1) + '*' * (2 * i + 1))

# Set the number of rows for the pyramid
num_rows = 5
print_pattern(num_rows)

```
Code execution approval requested:
```python
def print_triangle(rows):
    for i in range(rows):
        print(' ' * (rows - i - 1) + '*' * (2 * i + 1))

# Set the number of rows for the triangle
num_rows = 5
print_triangle(num_rows)

```

Coding Session:

[user]:
Write a Python function to display a triangle with string characters.

[code_executor]:
    *
   ***
  *****
 *******
*********


[coder]:
Great! The triangle has been displayed successfully! Here’s the function that creates the triangle using string characters:

```python
def print_triangle(rows):
    for i in range(rows):
        print(' ' * (rows - i - 1) + '*' * (2 * i + 1))

# Set the number of rows for the triangle
num_rows = 5
print_triangle(num_rows)
```

This code generates a symmetric

## 9. Clean Up

Always remember to close the model client when done:

In [8]:
# Close the model client connection
await model_client.close()
print("Model client closed successfully.")

Model client closed successfully.


## Summary

In this quickstart notebook, you've learned:

1. ✅ How to install and set up AutoGen
2. ✅ Creating basic agents with AssistantAgent
3. ✅ Running simple tasks with agents
4. ✅ Adding tools to make agents more capable
5. ✅ Creating multi-agent conversations
6. ✅ Streaming agent responses for better UX
7. ✅ Code execution with agents

## Next Steps

In the next notebook, we'll dive deeper into:
- Different types of agents
- Agent communication patterns
- Message handling and routing
- Custom agent creation

## Exercises

Try these exercises to reinforce your learning:

1. **Exercise 1**: Create an agent that can do math calculations
2. **Exercise 2**: Build a three-agent team (planner, executor, reviewer)
3. **Exercise 3**: Add a custom tool that fetches data from an API
4. **Exercise 4**: Create a chatbot that maintains conversation history